<a href="https://colab.research.google.com/github/maniparimi/langgraph/blob/main/langgraphrag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain
pip install langgraph
pip install cassandra-driver

In [ ]:
import cassio
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [ ]:
pip install langchain_community
pip install tiktoken
pip install langchain_groq
pip install langchain_hub
pip install langgraph
pip install langchain_huggingface

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
urls = [
    'https://example.com/agent',
    'https://example.com/prompt-engineering',
    'https://example.com/adversarial-attack-llm'
]
docs = [WebBaseLoader(url).load() for url in urls]

In [ ]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=0)
doc_splits = text_splitter.split_documents(docs)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', api_token=HF_TOKEN)

In [ ]:
from langchain.vectorstores.cassandra import Cassandra
astra_vector_store = Cassandra(embedding=embeddings, table_name='qa_mini_demo', session=None)

In [ ]:
astra_vector_store.add_documents(doc_splits)
print(f'Inserted {len(doc_splits)} records')

In [ ]:
retriever = astra_vector_store.as_retriever()
result = retriever.invoke('What is an agent?')
print(result)

In [ ]:
pip install wikipedia
pip install langchain_community

In [ ]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
wiki = WikipediaAPIWrapper(top_k_results=1, max_doc_content_chars=200)

In [ ]:
wiki.run('Tell me about Shahrukh Khan')

In [ ]:
from typing import Literal
from pydantic import BaseModel, Field
class RootQuery(BaseModel):
    data_source: Literal['vector_store', 'wiki_search']

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(api_key=GROQ_API_KEY, model_name='llama-3-70b-versatile')

In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are an expert at routing a user question to vector store or Wikipedia. The vector store contains documents related to agents, prompt engineering, and adversarial attacks. Use the vector store for questions on these topics. Otherwise, use wiki search.'),
    ('human', '{question}')
])

In [ ]:
from typing import Dict, List
class GraphState(Dict):
    question: str
    generation: str
    documents: List[str]

In [ ]:
def retrieve(state):
    question = state['question']
    documents = retriever.invoke(question)
    return {'documents': documents, 'question': question}

In [ ]:
def wiki_search(state):
    question = state['question']
    wiki_result = wiki.run(question)
    return {'documents': [wiki_result], 'question': question}

In [ ]:
def router(state):
    source = question_router.invoke(state['question'])
    if source.data_source == 'wiki_search':
        return 'wiki_search'
    else:
        return 'vector_store'

In [ ]:
from langgraph.graph import StateGraph
workflow = StateGraph(GraphState)
workflow.add_node('wiki_search', wiki_search)
workflow.add_node('retrieve', retrieve)
workflow.add_conditional_edges('start', router, {'wiki_search': 'wiki_search', 'vector_store': 'retrieve'})
workflow.add_edge('retrieve', 'end')
workflow.add_edge('wiki_search', 'end')
app = workflow.compile()

In [ ]:
from IPython.display import display
app.get_graph().draw_mermaid_png()

In [ ]:
for output in app.stream({'question': 'What is an agent?'}):
    for key, value in output.items():
        print(key, value[0]['metadata']['description'])